In [20]:
import requests
from scrapy.selector import Selector
import re
import os
from dotenv import load_dotenv
import telegram
import time 
from functools import wraps
load_dotenv()
token = os.getenv('token')
chat_id = os.getenv('chat_id')
user_agent = os.getenv('user_agent')

In [21]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [162]:
# список аниме

In [15]:
def get_series_cnt(content:Selector)->dict:
    text = "".join(content.xpath("./text()").getall())
    series_match= re.search(r"(\d+)\sсери", text)
    films_match = re.search(r"(\d+)\sфильм", text)
    series_cnt = int(series_match.group(1)) if series_match else 0
    films_cnt = int(films_match.group(1)) if films_match else 0
    return {'series_cnt': series_cnt,'films_cnt': films_cnt}
    
#series_cnts = [get_series_cnt_updated(item) for item in x_series]

In [16]:
#Selection
#selected = ['/full-metal-alchemist/']
#selected_dict = {key:item for key, item in anime_dict.items() if key in selected}

In [22]:
@timeit
def get_data(url:str)->dict:
    """The function gets url to scrape site content specifically title name,title page source, count of series,count of films'
    from all content pages of site and return dict object
    INPUT:
    URL:url adress must be like https://jut.su/anime/page-(\d)/
    Note: It's also use global constants from .env file  
    
    OUTPUT:
    dict: the dict object which kind is {title_internal_link:
    {'title_internal_link':title internal link,
     'title': title name in Russian,
     'series_cnt': series count,
     'film_cnt': film count
      }}"""
    anime_dict = {}
    page_num = 1
    while True:
        try:
            page_url = url + f"page-{page_num}/"
            response = requests.get(page_url,headers={
            "User-Agent": user_agent # global var
                    })
            content = response.content.decode("cp1251").encode('utf-8')
            selections = Selector(text=content).xpath('//div[contains(@class, "all_anime_global")]')
            links = selections.xpath('./a/@href').getall()
            titles_name = selections.xpath('.//div[contains(@class, "aaname")]/text()').getall()
            titles_info = selections.xpath('.//div[contains(@class, "aailines")]')
            series_cnts = [get_series_cnt(item) for item in titles_info]
            if len(series_cnts)==0:
                raise Exception
            for link, title, series_cnt in zip(links,titles_name,series_cnts):
                 if link not in anime_dict:
                        anime_dict[link] = {"ref":link, "title":title, "series_cnt":series_cnt['series_cnt'],'films_cnt':series_cnt['films_cnt']}   
            page_num+=1
            del series_cnts
        except Exception as e: 
            print(e)
            return (anime_dict or "No conection to the server")

In [114]:
# for key in selected:
#     response = requests.get(f"https://jut.su{key}",
#                        headers = {
#         'User-Agent': user_agent
#     })
#     content = response.content.decode('cp1251').encode('utf8')
#     series = Selector(text=response.content).xpath('//a[contains(@class, "short-btn")]//@href')
#     diff = len(series) - anime_dict[key]["n_series"]
#     if  diff!= 0:
#         name = anime_dict[key]["name"]
#         print(f"{name} {diff} новых серий!")
#     anime_dict[key]["n_series"] = len(series)
# # TODO descrpition
# # x.xpath('.//div[contains(@class, "tooltip_of_the_anime")]/@content').get()

In [10]:
# class AnimeObserver():
#     def __init__(self, selected):
#         self.selected = selected
        

In [11]:
selected_item = selected_dict['/full-metal-alchemist/']

In [ ]:
# from telegram.ext.updater import Updater
# from telegram.update import Update
# from telegram.ext.callbackcontext import CallbackContext
# from telegram.ext.commandhandler import CommandHandler
# from telegram.ext.messagehandler import MessageHandler
# from telegram.ext.filters import Filters
#
# updater = Updater("token", use_context=True)
#
#
# def start(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Hello sir, Welcome to the Bot.Please write\
#         /help to see the commands available.")
#
# def help(update: Update, context: CallbackContext):
#     update.message.reply_text("""Available Commands :-
#     /add - Add selected series
#     /remove - Remove series from selected list
#     """)
#
#
# def add(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         f"{selected_item['name']}: {selected_item['series_cnt']}")
#
# def remove(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Remove items")
#
#
#
#
#
# def unknown(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Sorry '%s' is not a valid command" % update.message.text)
#
#
# def unknown_text(update: Update, context: CallbackContext):
#     update.message.reply_text(
#         "Sorry I can't recognize you , you said '%s'" % update.message.text)
#
#
# updater.dispatcher.add_handler(CommandHandler('start', start))
# updater.dispatcher.add_handler(CommandHandler('add', add))
# updater.dispatcher.add_handler(CommandHandler('remove', remove))
# updater.dispatcher.add_handler(CommandHandler('help', help))
# updater.dispatcher.add_handler(MessageHandler(Filters.text, unknown))
# updater.dispatcher.add_handler(MessageHandler(
#     Filters.command, unknown)) # Filters out unknown commands
#
# # Filters out unknown messages.
# updater.dispatcher.add_handler(MessageHandler(Filters.text, unknown_text))
#
# updater.start_polling()
